In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib as mpl
import gseapy
from sklearn.decomposition import PCA
from sklearn import decomposition
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from kneed import KneeLocator
import ppscore as pps
import pathlib
import os
sc.settings.set_figure_params(dpi=80,dpi_save=300, color_map='viridis')

In [ ]:
adata = sc.read('/home/jovyan/YS_project/YS/Data_objects/final_objects/A4_V7_YS_integrated_data_singlets_with_raw_counts_for_MS_plotting_20211111_with_obsp.h5ad')

dict1 = {
'CS10' : 'CS10-11',
'CS11' : 'CS10-11',
'CS14' : 'CS14-15',
'CS15' : 'CS14-15',
'CS18' : 'CS18',
'CS22' : 'CS22-23',
'CS23' : 'CS22-23'
}

adata.obs['time_points'] = adata.obs['stage'].map(dict1)

# macrophage, erythroid, endoderm
mac_csv = pd.read_csv('/home/jovyan/YS_project/YS/YS_check_ealy_vs_late_csvs/sig_FC_early_late_neighbourhoods/Macrophage_sig_fc_early_late.csv', index_col = 1)
eryth_csv = pd.read_csv('/home/jovyan/YS_project/YS/YS_check_ealy_vs_late_csvs/sig_FC_early_late_neighbourhoods/Erythroid_sig_fc_early_late.csv', index_col = 1)
endo_csv = pd.read_csv('/home/jovyan/YS_project/YS/YS_check_ealy_vs_late_csvs/sig_FC_early_late_neighbourhoods/Endoderm_sig_fc_early_late.csv', index_col = 1)

#mac_csv = mac_csv[~mac_csv['nhood_Sig_FC'].str.match('False')]
#eryth_csv = eryth_csv[~eryth_csv['nhood_Sig_FC'].str.match('False')]
#endo_csv = endo_csv[~endo_csv['nhood_Sig_FC'].str.match('False')]

In [ ]:
adata_index = list(adata.obs.index)

n = 0
data_names = ['mac_csv','eryth_csv','endo_csv']

for i in [mac_csv,eryth_csv,endo_csv]:
    index_list = i.index[i.index.str.contains('Wang')]
    diff = list(set(index_list).difference(adata_index))
    for index in diff:
        i = i.rename(index={index: index+'_1'})
    index_list = i.index[i.index.str.contains('Wang')]
    intersect = (set(adata_index) & set(index_list))
    if len(intersect) == len(index_list):
        name = data_names[n]
        globals()[name] = i
        n +=1
        pass
    else:
        diff = list(set(index_list).difference(adata_index))
        for index in diff:
            i = i.rename(index={index: index+'_2'})
        index_list = i.index[i.index.str.contains('Wang')]
        intersect = (set(adata_index) & set(index_list))
        if len(intersect) == len(index_list):
            globals()[name] = i
            n +=1
            pass
        else:
            raise Exception(f'Error in barcode inetersect on loop {n}!')

In [ ]:
macrophage = adata[adata.obs.index.isin(list(mac_csv.index))][:]
erythroid = adata[adata.obs.index.isin(list(eryth_csv.index))][:]
endoderm = adata[adata.obs.index.isin(list(endo_csv.index))][:]

macrophage.obs['nhood_Sig_FC'] = mac_csv['nhood_Sig_FC'][mac_csv.index.isin(list(macrophage.obs.index))]
erythroid.obs['nhood_Sig_FC'] = eryth_csv['nhood_Sig_FC'][eryth_csv.index.isin(list(erythroid.obs.index))]
endoderm.obs['nhood_Sig_FC'] = endo_csv['nhood_Sig_FC'][endo_csv.index.isin(list(endoderm.obs.index))]

In [ ]:
objects = [macrophage,erythroid,endoderm]

for i in objects:
    i.obs['early_late_time_points'] = i.obs['nhood_Sig_FC'].astype(str) + '_' + i.obs['time_points'].astype(str)

In [ ]:
endoderm = endoderm[~endoderm.obs['early_late_time_points'].isin(['Late_CS14-15']),:]

In [ ]:
datas = [macrophage,endoderm]
names = ['Macrophage','Endoderm']
orig_name = ['macrophage','endoderm']
n = 0

for i in datas:
    name = names[n]
    temp_data = adata[adata.obs['broad_cell_labels_organ'].isin([name+'_ys'])]
    temp_data = temp_data[temp_data.obs['stage'].isin(['CS22','CS23'])]
    datas = [i,temp_data]
    new_data = sc.AnnData.concatenate(*datas, join='inner', batch_categories=None ,index_unique=None)
    del new_data.obs['batch']
    old_name = orig_name[n]
    globals()[old_name] = new_data
    n+=1

macrophage.obs.loc[macrophage.obs['time_points'].isin(['CS22-23']),['nhood_Sig_FC','early_late_time_points']] = ['Late','Late_CS22-23']
endoderm.obs.loc[endoderm.obs['time_points'].isin(['CS22-23']),['nhood_Sig_FC','early_late_time_points']] = ['Late','Late_CS22-23']

In [ ]:
objects = [macrophage,erythroid,endoderm]
objects_name = ['macrophage','erythroid','endoderm']

In [ ]:
cell_cycle_genes = pd.read_csv('/home/jovyan/Useful code and scripts/issac_cell_cycle_and_respiration/GO-0022402_cell_cycle_genes (1).csv', index_col=0, header=None)
cell_respr_genes = pd.read_csv('/home/jovyan/Useful code and scripts/issac_cell_cycle_and_respiration/GO-0045333_cell_respiration (1).csv', index_col=0, header=None)
cell_cycle_genes_list = list(cell_cycle_genes.index)
cell_respr_genes_list = list(cell_respr_genes.index)

In [ ]:
list_to_remove = []

for i in objects:
    remove = list(i.var.index[i.var_names.str.match("HB|MT-|RPS|RPL")])
    list_to_remove = list_to_remove + remove + cell_cycle_genes_list + cell_respr_genes_list
    list_to_remove = list(dict.fromkeys(list_to_remove))

In [ ]:
n = 0
for x in objects:
    x = x[:,~x.var_names.isin(list_to_remove)]
    name = objects_name[n]
    globals()[name] = x
    n += 1

In [ ]:
objects = [macrophage,erythroid,endoderm]

for i in objects:
    print('Before pre-processing shape:')
    print(i.shape)
    #ud.describe_basic(i)
    sc.pp.filter_genes(i, min_cells=3)
    sc.pp.normalize_total(i)
    sc.pp.log1p(i)
    sc.pp.highly_variable_genes(i)
    print('After pre-processing shape:')
    #ud.describe_basic(i)
    print(i.shape)
    print('Loop complete')
    print('')

In [ ]:
# Variables
Object = endoderm
key_var = 'early_late_time_points'
run_name = 'Endoderm_Early_Late_across_time_DE'
save_loc = '/lustre/scratch117/cellgen/team298/SharedFolders/Collab_scripting/GSEA_confusion_mat/Early_late_across_time_outs_20220224'

# Functions

In [ ]:
def setup(save_loc):
    save_loc = save_loc + datetime.date.today().strftime("_%I%M_%d%m%Y")
    if not os.path.exists(save_loc):
        os.makedirs(save_loc)
    adata = sc.read(Object)
    adata.var_names_make_unique()
    return save_loc, adata

def setup2(save_loc):
    save_loc = save_loc
    adata = Object
    adata.var_names_make_unique()
    return save_loc, adata

def Pre_process(adata,key_var,top_de = 50):
    if np.max(adata.X) > 1000:
        sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
        sc.pp.log1p(adata)

    spot_var = key_var
    DE_name = (save_loc+ '/' + run_name + '_DEGS.csv')
    sc.tl.rank_genes_groups(adata, spot_var, method='wilcoxon',n_genes=500)
    result = adata.uns['rank_genes_groups']
    groups = result['names'].dtype.names

    DE = pd.DataFrame(
        {group + '_' + key[:1]: result[key][group]
        for group in groups for key in ['names', 'pvals','logfoldchanges']}).head(500)
    #DE_name ="./DEGS_gastrulation_211121.csv"
    DE.to_csv(DE_name)
    degs = DE[:]
#     if 'concat' in locals() or 'concat' in globals():
#         del(concat)
    n = degs.loc[:, degs.columns.str.endswith("_n")]
    n = pd.melt(n)
    p = degs.loc[:, degs.columns.str.endswith("_p")]
    p = pd.melt(p)
    l = degs.loc[:, degs.columns.str.endswith("_l")]
    l = pd.melt(l)
    n = n.replace(regex=r'_n', value='')
    n = n.rename(columns={"variable": "cluster", "value": "gene"})
    p = (p.drop(["variable"],axis = 1)).rename(columns={ "value": "p_val"})
    l = (l.drop(["variable"],axis = 1)).rename(columns={ "value": "logfc"})
    # Rank Default DEGs with::
    concat = pd.concat([n,p,l],axis=1)
    #remove mito genes from data
    concat = concat[~concat["gene"].str.startswith("MT-")]
    concat = concat[~concat["gene"].str.startswith("RP11-")]
    concat["cluster"] = concat["cluster"].astype(str)
    marker_df = concat.groupby('cluster').apply(lambda grp: grp.nsmallest(top_de*2, 'p_val')).reset_index(drop=True)
    marker_df = marker_df.groupby('cluster').apply(lambda grp: grp.nlargest(top_de, 'logfc')).reset_index(drop=True)
    markers = marker_df.groupby('cluster')['gene'].apply(list).to_dict()
    markers = dict(sorted(markers.items()))
    markers = {str(k):v for k,v in markers.items()}
    markers = markers
    sc.pl.dotplot(adata, var_names = markers, groupby=spot_var,dendrogram=False,standard_scale='var', color_map='Reds', show = True, save = (run_name + '.pdf'))
    return marker_df, markers, adata

# Naively submit all DEGs
def gseapy_enr(marker_df):
    import gseapy
    gene_set_names = gseapy.get_library_name(organism='Human')

    for i in list(marker_df['cluster'].unique()):
        #Available databases : 'Human', 'Mouse', 'Yeast', 'Fly', 'Fish', 'Worm' 
        import gseapy
        gene_set_names = gseapy.get_library_name(organism='Human')
        glist = list(marker_df['gene'][marker_df['cluster'].isin([i])])
        enr_res = gseapy.enrichr(gene_list=glist,
                         organism='Human',
                         gene_sets='GO_Biological_Process_2021' ,#'GO_Molecular_Function_2021', #''GO_Biological_Process_2021',
                         description='pathway',
                         cutoff = 0.05)
        enr_res.res2d.to_csv(save_loc + '/'+i+'_enr_res_score.csv')
        
def generate_cluster_maps():
    Gene_set_dict_per_category = {}
    gsea_paths = pd.DataFrame(pathlib.Path(save_loc).glob("*res_score.csv"),columns = ['fpaths'])
    for gsea_fpath in gsea_paths['fpaths']:
        enr_name = str(gsea_fpath).replace(save_loc+'/','')
        enr_name = str(enr_name).replace('.csv','')
        gsea = pd.read_csv(gsea_fpath,index_col=0)
        gsea = gsea[gsea['P-value'] < 0.05]
        gsea['g_list'] = 'NAN'
        gsea_dict = {}
        for z in gsea.index:
            term = (gsea['Term'][gsea.index==z].item())
            gsea_dict[term] = (gsea.loc[gsea.index == z ,'Genes'][z]).split(";")
            if len(gsea_dict[term]) >= min_term:
                gsea.at[z,'g_list'] = gsea_dict[term]
        rows_to_remove = list(gsea.index[gsea['g_list']== 'NAN'])
        gsea = gsea[~gsea.index.isin(rows_to_remove)]
        unique_genes = list(np.unique(np.array([item for sublist in gsea['g_list'].values for item in sublist])))
        gseapy_matrix = pd.DataFrame(columns=gsea.Term.unique(), index=unique_genes)
        for col in gseapy_matrix.columns:
            for index in gseapy_matrix.index:
                if index in gsea_dict[col]:
                    gseapy_matrix.at[index,col] = 1 # this is not a clean way to do this // rewrite
                else:
                    gseapy_matrix.at[index,col] = 0 # this is not a clean way to do this // rewrite
        for i in gseapy_matrix.columns:
            gseapy_matrix[i] = gseapy_matrix[i].astype('int')
        # future updte Let's use PPI matrix instead of corr as this gives better predictive value
        # corr_table = pps.matrix(gseapy_matrix)
        # corr_table = corr_table.pivot(columns='x', index='y', values='ppscore')
        corr_table = gseapy_matrix.corr()
        
        # simple clustermap just on corr_table
        cm = sns.clustermap(corr_table, figsize=(30,30))
        plt.setp(cm.ax_heatmap.xaxis.get_majorticklabels(), fontsize=20)
        plt.setp(cm.ax_heatmap.yaxis.get_majorticklabels(), fontsize=20)
        plt.title(enr_name)
        
        
        
        # Neighbourhoods clustergram
        pca = PCA(n_components=int(len(gsea)/2))
        pca.fit(corr_table)
        X_pca = pca.transform(corr_table)
        X = X_pca
        y = pd.DataFrame(X_pca).index
        sil_score = []
        n_clust = [] 
        models = {}
        if len(gsea) < 10:
            for n_cluster in range(2, len(gsea)):
                kmeans = KMeans(n_clusters=n_cluster).fit(X)
                models[n_cluster] = kmeans
                label = kmeans.labels_
                sil_coeff = silhouette_score(X, label, metric='euclidean')
                n_clust.append(n_cluster)
                sil_score.append(sil_coeff)
        else:
            for n_cluster in range(2, 11):
                kmeans = KMeans(n_clusters=n_cluster).fit(X)
                models[n_cluster] = kmeans
                label = kmeans.labels_
                sil_coeff = silhouette_score(X, label, metric='euclidean')
                n_clust.append(n_cluster)
                sil_score.append(sil_coeff)
                
        kneedle = KneeLocator(x = sil_score, y = n_clust, S=0.01, curve="convex", direction="increasing")
        model = models[n_clust[sil_score.index(kneedle.knee)]]
        pred_clusters = model.predict(X)
        figure, ax = plt.subplots(figsize=(5, 5))
        ax = plt.scatter(X[:,0], X[:,1], c=pred_clusters)
        plt.legend(handles=ax.legend_elements()[0], labels=list(set(pred_clusters)), bbox_to_anchor=(1.05, 1))
        plt.title(enr_name)
        gsea['k_clust'] = pred_clusters
        gsea = gsea.sort_values(by=['k_clust'])
        
        #Filter junk clusters
        ranker = pd.DataFrame(gsea.groupby('k_clust').count().iloc[:,0]).sort_values(pd.DataFrame(gsea.groupby('k_clust').count().iloc[:,0]).columns[0],ascending=False).head(5)
        gsea = gsea[gsea['k_clust'].isin(ranker.index)]
        corr_table = corr_table[corr_table.index.isin(gsea['Term'])]
        corr_table = corr_table[(gsea['Term'])]
        corr_table = corr_table.reindex(list(gsea['Term']))
        corr_table = corr_table[list(gsea['Term'])]
        corr_table_organised = corr_table[:]
        corr_table_organised.index = 'clus_'+ gsea['k_clust'].astype(str)+ '_' +corr_table_organised.index.astype(str) 
        
        pal =sns.color_palette("magma", as_cmap=True)# sns.diverging_palette(10, 240, n=20)
        figure, ax = plt.subplots(figsize=(30, 30))
        ax = sns.heatmap(corr_table_organised, annot=False,vmin=min(np.min(corr_table)), vmax=1, linewidths=1, center=np.median(np.min(corr_table)), square=True, annot_kws={"size": 16}) # font size
        ax.set_title(enr_name)
        figure.savefig((save_loc+'/'+enr_name+'_corr_plot.pdf'),bbox_inches='tight',dpi=300)  
        gsea.to_csv(save_loc+'/'+enr_name+'_enr_score_k_clust.csv')
        corr_table_organised.to_csv(save_loc+'/'+enr_name+'_corr.csv')
        
        # Dictionary
        dict_name = enr_name.replace('res_score','clust_dict')
        key_name = enr_name.replace('res_score','')
        Gene_set_clust_dict = {}
        
        for clust in gsea['k_clust'].unique():
            key = key_name + 'Gene_set_' + str(clust)
            subset = gsea[gsea['k_clust'].isin([clust])]
            values = []
            for index in subset.index:
                genes = (subset.loc[subset.index == index ,'Genes'][index]).split(";")
                [values.append(gene) for gene in genes if gene not in values]
            Gene_set_clust_dict[key] = values
        globals()[dict_name] = Gene_set_clust_dict # Returns individual dictionary for the category e.g. if category = 'Early': Early_enr_clust_dict
        Gene_set_dict_per_category.update(Gene_set_clust_dict)
    cwd = os.getcwd()
    os.chdir(save_loc)
    sc.pl.dotplot(adata, Gene_set_dict_per_category, groupby=key_var, use_raw=False, standard_scale='var', save='gseapy_geneset_overall_cluster_dotplot.pdf')
    os.chdir(cwd)
    globals()['Gene_set_dict_per_category'] = Gene_set_dict_per_category

In [ ]:
top_de = 15
save_loc, adata = setup2(save_loc)
marker_df, markers, adata = Pre_process(adata,key_var, top_de = top_de)
gseapy_enr(marker_df)
min_term = 2
generate_cluster_maps()

In [ ]:
endoderm2 = endoderm2[~endoderm2.obs['nhood_Sig_FC'].str.contains('False')]
sc.pp.scale(endoderm2, zero_center=True)

In [ ]:
plot_dict = {

'Fibrinogen_and_cross_linking': ['FGB','FGG','FGA'],

'Retinoid_metabolic_process': ['RBP4','RBP2',  'RBP3', 'RARRES2'],  
    
'Lipid metabolic processes':['APOM','APOB','ALB','FABP1','FASN', 'CYP51A1','LSS','GPAM','SREBF1',],
 
'Regulation of apoptotic process': ['NFKBIA','JUN','ATF3',],
    
}

sc.pl.matrixplot(endoderm2, var_names=plot_dict, use_raw=False, groupby='early_late_time_points', vmax=1, vmin=-1, cmap='RdBu_r', save='early_late_degs_cut_20220323.pdf', colorbar_title='mean z-score',)